# 
####    :
> ###### 
> ###### 
---
####    
> ###### t]
> ######
> ###### 
---
> ###### 
> ###### 
> ###### 
---


#### 4.   

In [1]:
#Colab資料夾設定方法
#!pip install google.colab #如未安裝取消註解後執行
import logging
import os
import pandas as pd
import numpy as np
from numpy import asarray 
from numpy import zeros
import re
#from google.colab import drive

#drive.mount('/content/drive')

##出現提示欄進行授權

#os.chdir('/content/drive//My Drive/KCC NLP 2021 Test/data') #切換該目錄
#os.listdir() #確認目錄內容

In [2]:
#讀檔及文字處理
#read source data from labeled file

df = pd.read_csv(r"/Users/jojoteng/Downloads/data/標記檔/第十屆綠標記.csv")
print("length of csv file = ", len(df))
print("-----------------------------------------------------------")

#僅保留情緒標記及內容欄位
df = df[['情緒標記','內容']]
print("length after fields retrieving: ", len(df))

#變更欄位名稱
df = df.rename(columns={"情緒標記":"Label", "內容":"Text"})

# 濾除標點及空白雜物等
def replace_all_blank(value):
# \W 表示匹配非數字字母下劃線
    return re.sub('\W+', '', value).replace("_", '')

df["Text"] = df["Text"].apply(replace_all_blank)
print(df.head(5))

#過濾重複及含空白資料
df = df.drop_duplicates().copy()
df = df.dropna()
print("length after droping duplicate and Null = ", len(df))

#強制將情緒標記欄位轉成整數
def apply_int(x):
    return int(x)
    
df["Label"] = df["Label"].apply(apply_int)


#僅保留 "情緒標記" 為 1 至 5 資料;
#情緒值: １（指責）～３（無法判斷）～５（嘉許）
df = df[(df["Label"] > 0) & (df["Label"] < 6)]
print(" 情緒標記 介於 1-5 筆數: ", len(df))


# ---------------------------------------------- 情緒標記轉換---------------------------------------------------
# 情緒標記轉換V1:
print("---------------------------------------情緒標記轉換 V1: 兩極法 (1-2: 負評(0) ; 4-5: 正評(1); 捨棄 3)--------------------------------")
dfv1 = df.copy()
dfv1 = dfv1[(dfv1["Label"] != 3)]
print(" 濾除 情緒標記 = 3 資料後筆數: ", len(dfv1))

def sent_conv1(x):
    if x < 3:
        r = 0
    else:
        r = 1
    return int(r)

dfv1["Label"] = dfv1["Label"].apply(sent_conv1)
print(dfv1.head(5))

# 情緒標記轉換V2:
print("----------------------------------------情緒標記轉換 V2: 問責法 (1-2: 負評(0) ; 3-5: 無負評(1))--------------------------------")
dfv2 = df.copy()
def sent_conv2(x):
    if x < 3:
        r = 0
    else:
        r = 1
    return int(r)

dfv2["Label"] = dfv2["Label"].apply(sent_conv2)
print(dfv2.head(5))

# 情緒標記轉換V3:
print("----------------------------------------情緒標記轉換 V3: 採務實法(v3)：1-2: 負評(0) ; 3: 中性(1); 4-5: 正評(2) --------------------------------")
dfv3 = df.copy()
def sent_conv3(x):
    if x < 3:
        r = 0
    elif x == 3:
        r = 1
    else:
        r = 2
    return int(r)

dfv3["Label"] = dfv3["Label"].apply(sent_conv3)
print(dfv3.head(5))

length of csv file =  1547
-----------------------------------------------------------
length after fields retrieving:  1547
   Label                                               Text
0      2  今天蘇院長拍板建立國產口罩產線預計一個月後日產一千萬片但嘉瑜還是要呼籲產線建立前買口罩不該讓...
1      2  一請政府儘速建立身分認證機制確保依照限購政策人人都買得到口罩或由鄰里長協助派發口罩二中央地方...
2      3  面對武漢肺炎疫情竟然還在造謠政治操作韓粉造謠者已經在一天內被逮捕警方研判有指使者正在進一步追...
3      3  同學家長們請注意開學延後因應中國武漢肺炎疫情升溫為維護師生健康教育部針對高級中等含以下學校延...
4      3  重大消息重大消息重大消息剛剛教育部潘文忠部長在防疫記者會上宣布台灣高中以下學校寒假將延長至2...
length after droping duplicate and Null =  1546
 情緒標記 介於 1-5 筆數:  1546
---------------------------------------情緒標記轉換 V1: 兩極法 (1-2: 負評(0) ; 4-5: 正評(1); 捨棄 3)--------------------------------
 濾除 情緒標記 = 3 資料後筆數:  455
    Label                                               Text
0       0  今天蘇院長拍板建立國產口罩產線預計一個月後日產一千萬片但嘉瑜還是要呼籲產線建立前買口罩不該讓...
1       0  一請政府儘速建立身分認證機制確保依照限購政策人人都買得到口罩或由鄰里長協助派發口罩二中央地方...
10      0  失控的中國肺炎疫情中共政權會怎麼因應世界各國包括台灣都要審慎嚴肅面對二個建議台灣政府應該思考...
14      1  請大家一起為站在第一線的防疫人員加油昨天陳時中部長在記者會落淚宣布武漢肺炎疫情台灣又多一例確

In [3]:

# 設定每筆資料最大長度,以配合 bert-base-chinese model 長度 512
MAX_LENGTH = 512

In [4]:
#----------- 濾除 Text長度大於 MAX_LENGTH 資料 ------------------------
# deep copy 各版已完成情緒標記轉換之資料
df_FV1 = dfv1.copy()
df_FV2 = dfv2.copy()
df_FV3 = dfv3.copy()

print("====================================================== FV1: 過濾版 + 情緒轉換 V1 =======================================================")
df_FV1 = df_FV1[(df_FV1["Text"].str.len() < MAX_LENGTH)]
print("FV1 濾除長度過大後筆數= ", len(df_FV1))

# 抽樣 80% 訓練, 20% 驗證
dfFV1_train = df_FV1.sample(frac=0.8, replace=False)
print("FV1 抽樣訓練筆數 = ", len(dfFV1_train))
print(dfFV1_train.head(5))
print("-----------------------------------------------------------")

dfFV1_verf = df_FV1.drop(dfFV1_train.index)
print("FV1 抽樣驗證筆數 = ", len(dfFV1_verf))

# 打亂資料排列, 增加訓練效果

dfFV1_train = dfFV1_train.sample(frac=1).reset_index(drop=True)
dfFV1_verf = dfFV1_verf.sample(frac=1).reset_index(drop=True)
print("---------FV1 確認訓練順序是否已打亂------------")
print(dfFV1_train.head(5))

# FV1 output: 濾除過長版
dfFV1_train.to_csv("LYTrain-Senti256FV1.csv", sep="\t", index=False,encoding='utf_8_sig')
dfFV1_verf.to_csv("LYVerf-Senti256FV1.csv", sep="\t", index=False,encoding='utf_8_sig')

print("====================================================== FV2: 過濾版 + 情緒轉換 V2 =======================================================")
df_FV2 = df_FV2[(df_FV2["Text"].str.len() < MAX_LENGTH)]
print("FV2 濾除長度過大後筆數= ", len(df_FV2))

# 抽樣 80% 訓練, 20% 驗證
dfFV2_train = df_FV2.sample(frac=0.8, replace=False)
print("FV2 抽樣訓練筆數 = ", len(dfFV2_train))
print(dfFV2_train.head(5))
print("-----------------------------------------------------------")

dfFV2_verf = df_FV2.drop(dfFV2_train.index)
print("FV2 抽樣驗證筆數 = ", len(dfFV2_verf))

# 打亂資料排列, 增加訓練效果
dfFV2_train = dfFV2_train.sample(frac=1).reset_index(drop=True)
dfFV2_verf = dfFV2_verf.sample(frac=1).reset_index(drop=True)
print("---------FV2 確認順序是否已打亂------------")
print(dfFV2_train.head(5))

# FV2 output: 濾除過長版
dfFV2_train.to_csv("LYTrain-Senti256FV2.csv", sep="\t", index=False,encoding='utf_8_sig')
dfFV2_verf.to_csv("LYVerf-Senti256FV2.csv", sep="\t", index=False,encoding='utf_8_sig')

print("====================================================== FV3: 過濾版 + 情緒轉換 V3 =======================================================")
df_FV3 = df_FV3[(df_FV3["Text"].str.len() < MAX_LENGTH)]
print("FV3 濾除長度過大後筆數= ", len(df_FV3))

# 抽樣 80% 訓練, 20% 驗證
dfFV3_train = df_FV3.sample(frac=0.8, replace=False)
print("FV3 抽樣訓練筆數 = ", len(dfFV3_train))
print(dfFV3_train.head(5))
print("-----------------------------------------------------------")

dfFV3_verf = df_FV3.drop(dfFV3_train.index)
print("FV3 抽樣驗證筆數 = ", len(dfFV3_verf))

# 打亂資料排列, 增加訓練效果
dfFV3_train = dfFV3_train.sample(frac=1).reset_index(drop=True)
dfFV3_verf = dfFV3_verf.sample(frac=1).reset_index(drop=True)
print("---------FV3 確認順序是否已打亂------------")
print(dfFV3_train.head(5))

# FV3 output: 濾除過長版
dfFV3_train.to_csv("LYTrain-Senti256FV3.csv", sep="\t", index=False,encoding='utf_8_sig')
dfFV3_verf.to_csv("LYVerf-Senti256FV3.csv", sep="\t", index=False,encoding='utf_8_sig')

====================================================== FV1: 過濾版 + 情緒轉換 V1 =======================================================
FV1 濾除長度過大後筆數=  385
FV1 抽樣訓練筆數 =  308
     Label                                               Text
141      0  又或者在我國學前教育公立幼兒園的教師是要上班的但若是教保員或是助理教保員因為他的雇傭關係是適...
590      1  琪銘今天就美國大選後兩岸關係質詢我特別要強調咱台灣絕對有我們的價值不論是我們的防疫成果科技成...
891      0  1報載檢警深入調查周男等共多達15人涉案澄清當日其中一人到警所探訪友人酒駕遭拘因而與丁男遇上...
366      0  剛才中國強勢通過了港版國安法正式宣告香港的自由民主不復存在阿惠要在這邊嚴厲譴責這個邪惡的極權...
400      0  部份婦女因先天問題或重大疾病無法懷孕生產可惜受限於國內法令這些家庭不是放棄生育下一代的夢想就...
-----------------------------------------------------------
FV1 抽樣驗證筆數 =  77
---------FV1 確認訓練順序是否已打亂------------
   Label                                               Text
0      1  今天一早秉叡與蘇貞昌院長及衛生福利部陳時中部長到我們新莊的一安藥局參訪除了要向一線的藥師藥生...
1      1  看了會感動的日本傑尼斯天團嵐成員櫻井翔主持的教科書學不到的超級難題節目中昨4日釋出台灣防疫特...
2      1  今天立法院經濟委員會審查110年度經濟部與所屬單位預算從早上審到下午6小時連續馬拉松式的審查...
3      0  昨天立法院邀請教育部報告青年職涯發展政策規劃惠員以如何擴大辦理大專校院農業公費專班檢討青年儲...
4      1  今天蘇貞昌院長與各部會召開記者會向國人同胞報告未來將提出第二階段紓困方案

In [2]:
# 1+1

SyntaxError: invalid syntax (903597241.py, line 1)